In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Reading our data
df = pd.read_csv("/kaggle/input/netflix-shows/netflix_titles.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
#Checking for unique values
df.nunique()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Helps to visualize missing values in each columns
import missingno as msno

In [ ]:
# Showing NaN values by using heatmap
msno.heatmap(df)

# Cleaning of data

In [ ]:
#There is only seven NaN values in rating, so I'll just put values.
df[df['rating'].isna()]

In [ ]:
df.isnull().sum()

In [ ]:
changing_nan = {
    67: 'TV-PG',
    2359: 'TV-14',
    3660: 'TV-MA',
    3736: 'TV-MA',
    3737: 'NR',
    3738: 'TV-MA',
    4323: 'TV-MA '
}

for id, rate in changing_nan.items():
    df.iloc[id, 6] = rate
    
df.isnull().sum()

In [ ]:
#Now we will be dropping Null values from Cast!
df = df[df['cast'].notna()]
df['cast'].isna().sum()

In [ ]:
import collections

In [ ]:
#Now in country column, ill be replacing null value with the ost common country name which exist in our dataset!
collections.Counter(df['country']).most_common(1)

In [ ]:
#We can see that United States has maximum number of records!
df['country'] = df['country'].fillna('United States')
df['country'].isna().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
pip install xlrd

In [ ]:
import pandas as pd
!pip install AutoViz
from autoviz.AutoViz_Class import AutoViz_Class

# Visualizaing our data!

In [ ]:
from autoviz.AutoViz_Class import AutoViz_Class

In [ ]:
#EDA using Autoviz
autoviz = AutoViz_Class().AutoViz("../input/netflix-shows/netflix_titles.csv")

In [ ]:
#Which are the top oldest movies on Nextflix!??
old = df.sort_values("release_year", ascending = True)
old = old[old['duration'] != ""]
old = old[old['type'] !="TV Show"]
old[['title', "release_year","country","duration"]][:15]

In [ ]:
#Which are the top Newest movies on Nextflix!??
old = df.sort_values("release_year", ascending = False)
old = old[old['duration'] != ""]
old = old[old['type'] !="TV Show"]
old[['title', "release_year","country","duration"]][:15]

In [ ]:
import plotly.express as px

In [ ]:
#Country wise representation of all Movies!
country  = df['country'].value_counts()
country = country[:15,]
fig = px.bar(x=country.index, y=country.values, color=country.values,
             hover_data=[country.index, country.values],labels={'country':'Frequency'}, height=400)
fig.show()

# Recommender System:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(min_df = 3,max_features = None,analyzer = 'word',token_pattern = 'r\w{1,}', ngram_range = (1,3), stop_words = 'english')

In [ ]:
df.head()

In [ ]:
df['combined'] = df['description']+df['cast']+df['director']
df.head(5)

In [ ]:
df['combined'] = df['combined'].fillna("")

In [ ]:
tfv_matrix = tfv.fit_transform(df['combined'])
from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix,tfv_matrix)
sig[0]
indices = pd.Series(df.index,index = df['title']).drop_duplicates()
indices

In [ ]:
def recommend(title,sig=sig):
    idx = indices[title]
    sig_scores = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_scores,key = lambda x:x[1], reverse = True)
    sig_scores = sig_scores[1:11]
    movies_indices = [i[0] for i in sig_scores]
    return df['title'].iloc[movies_indices]

In [ ]:
recommend("Payday")

In [ ]:
recommend('Inside Man: Most Wanted')

# Your suggestion will be highly appreicated.Please upvote if you like it.